In [1]:
%load_ext nb_black
# !pip install nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu

# import PPOCS as p
from sklearn.utils import shuffle
import math

import warnings

warnings.filterwarnings("ignore")
randomseed = 7
np.random.seed(randomseed)

Using TensorFlow backend.


<IPython.core.display.Javascript object>

# 1. Read the dataset

In [3]:
x_original = pd.read_csv("XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

print(x_original.shape)

(48095, 20)


<IPython.core.display.Javascript object>

In [4]:
x_original2 = pd.read_csv("XLable_onlyDiabeticRemoved2.txt")

conditions = [
    (x_original2.L100800 < 100),
    (x_original2.L100800 >= 100) & (x_original2.L100800 < 126),
    (x_original2.L100800 >= 126),
]
choices = [0, 1, 2]
x_original2["CLASS"] = np.select(conditions, choices, default=0)
x_original2 = x_original2[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

print(x_original2.shape)

(48095, 20)


<IPython.core.display.Javascript object>

In [5]:
x_original3 = pd.read_csv("XLable_onlyDiabeticRemoved3.txt")

conditions = [
    (x_original3.L100800 < 100),
    (x_original3.L100800 >= 100) & (x_original3.L100800 < 126),
    (x_original3.L100800 >= 126),
]

choices = [0, 1, 2]
x_original3["CLASS"] = np.select(conditions, choices, default=0)

x_original3 = x_original3[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

print(x_original3.shape)

(48095, 20)


<IPython.core.display.Javascript object>

In [6]:
y_original = pd.read_csv("TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "FIELD_1", "CLASS"]]

print(y_original.shape)

(48095, 3)


<IPython.core.display.Javascript object>

In [7]:
data = pd.merge(
    x_original,
    x_original2,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)
data.head()

,Unnamed: 0,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,...,FIELD_40_y,FIELD_31_y,SEX_y,AGE_y,FIELD_16_y,FIELD_23_y,FIELD_15_y,FIELD_22_y,FIELD_17_y,FIELD_24_y
0,0,77.0,NaN,53.0,20.1,13.0,3.0,1.0,1.0,0.0,...,1.0,0.0,1.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,98.0,NaN,80.0,24.7,13.0,4.1,1.0,0.0,0.0,...,0.0,1.0,1.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100.0,NaN,56.0,22.1,15.0,4.0,1.0,0.0,0.0,...,2.0,1.0,1.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,78.0,NaN,92.0,23.5,13.0,4.3,1.0,0.0,3.0,...,3.0,0.0,1.0,68.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,92.0,NaN,67.0,23.2,13.0,4.7,1.0,0.0,3.0,...,4.0,0.0,1.0,69.0,0.0,0.0,0.0,0.0,1.0,1.0


<IPython.core.display.Javascript object>

In [8]:
data = pd.merge(
    data,
    x_original3,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)
data.head()

,Unnamed: 0,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,...,FIELD_40,FIELD_31,SEX,AGE,FIELD_16,FIELD_23,FIELD_15,FIELD_22,FIELD_17,FIELD_24
0,0,77.0,NaN,53.0,20.1,13.0,3.0,1.0,1.0,0.0,...,1.0,0.0,1.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,98.0,NaN,80.0,24.7,13.0,4.1,1.0,0.0,0.0,...,1.0,0.0,1.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100.0,NaN,56.0,22.1,15.0,4.0,1.0,0.0,0.0,...,1.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,78.0,NaN,92.0,23.5,13.0,4.3,1.0,0.0,3.0,...,4.0,0.0,1.0,69.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,92.0,NaN,67.0,23.2,13.0,4.7,1.0,0.0,3.0,...,2.0,0.0,1.0,70.0,0.0,0.0,0.0,0.0,1.0,1.0


<IPython.core.display.Javascript object>

In [9]:
data = pd.merge(
    data,
    y_original,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)

<IPython.core.display.Javascript object>

In [10]:
data.head()

,Unnamed: 0,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,...,FIELD_31,SEX,AGE,FIELD_16,FIELD_23,FIELD_15,FIELD_22,FIELD_17,FIELD_24,CLASS
0,0,77.0,NaN,53.0,20.1,13.0,3.0,1.0,1.0,0.0,...,0.0,1.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,98.0,NaN,80.0,24.7,13.0,4.1,1.0,0.0,0.0,...,0.0,1.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,100.0,NaN,56.0,22.1,15.0,4.0,1.0,0.0,0.0,...,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,78.0,NaN,92.0,23.5,13.0,4.3,1.0,0.0,3.0,...,0.0,1.0,69.0,0.0,0.0,0.0,0.0,1.0,1.0,0
4,4,92.0,NaN,67.0,23.2,13.0,4.7,1.0,0.0,3.0,...,0.0,1.0,70.0,0.0,0.0,0.0,0.0,1.0,1.0,1


<IPython.core.display.Javascript object>

In [11]:
# filter the data set
data = data[data.FIELD_16_x != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23_x != 1]  # exclude people who are on medication for diabetes

data = data[data.FIELD_16_y != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23_y != 1]  # exclude people who are on medication for diabetes

data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

# data = data[
#     data.FIELD_15_x != 1
# ]  # exclude people who are diagnosed for (high blood pressure)
# data = data[
#     data.FIELD_22_x != 1
# ]  # exclude people who are on medication for high blood pressure

# data = data[
#     data.FIELD_15_y != 1
# ]  # exclude people who are diagnosed for (high blood pressure)
# data = data[
#     data.FIELD_22_y != 1
# ]  # exclude people who are on medication for high blood pressure

# data = data[data.FIELD_17_x != 1]  # exclude people who are diagnosed for hyperlipidemia
# data = data[
#     data.FIELD_24_x != 1
# ]  # exclude people who are on medication for hyperlipidemia

# data = data[data.FIELD_17_y != 1]  # exclude people who are diagnosed for hyperlipidemia
# data = data[
#     data.FIELD_24_y != 1
# ]  # exclude people who are on medication for hyperlipidemia

# print(data.shape)

<IPython.core.display.Javascript object>

In [12]:
data = data[
    [
        "L100800_x",
        "L104600_x",
        "L103000_x",
        "S000300_x",
        "L101700_x",
        "L100700_x",
        "FIELD_33_x",
        "FIELD_38_x",
        "FIELD_40_x",
        "FIELD_31_x",
        "SEX_x",
        "AGE_x",
        "L100800_y",
        "L104600_y",
        "L103000_y",
        "S000300_y",
        "L101700_y",
        "L100700_y",
        "FIELD_33_y",
        "FIELD_38_y",
        "FIELD_40_y",
        "FIELD_31_y",
        "SEX_y",
        "AGE_y",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()

(11360, 37)


,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,FIELD_31_x,...,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
13,84.0,5.0,84.0,19.9,11.0,5.1,1.0,1.0,1.0,0.0,...,20.4,10.0,5.1,1.0,1.0,2.0,0.0,1.0,21.0,0
14,86.0,5.3,125.0,20.3,9.0,5.3,1.0,2.0,2.0,0.0,...,20.0,7.0,4.9,1.0,1.0,3.0,0.0,1.0,22.0,0
15,81.0,4.9,46.0,18.4,14.0,4.1,1.0,2.0,0.0,0.0,...,18.8,12.0,5.4,1.0,3.0,0.0,0.0,1.0,21.0,0
16,70.0,4.9,35.0,18.6,11.0,5.4,1.0,1.0,0.0,0.0,...,18.9,15.0,4.8,1.0,4.0,0.0,0.0,1.0,22.0,0
17,90.0,5.2,118.0,18.7,10.0,3.5,1.0,1.0,0.0,0.0,...,18.9,8.0,4.0,1.0,0.0,0.0,0.0,1.0,22.0,0


<IPython.core.display.Javascript object>

In [13]:
data = shuffle(data)
data.head()

,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,FIELD_31_x,...,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
15320,96.0,5.30,238.0,25.1,149.0,7.5,2.0,3.0,2.0,0.0,...,23.8,95.0,6.1,2.0,0.0,0.0,0.0,0.0,44.0,1
7243,93.0,5.16,139.0,28.4,18.0,6.3,1.0,1.0,0.0,0.0,...,28.7,33.0,6.1,1.0,2.0,0.0,0.0,0.0,39.0,0
32596,108.0,5.60,150.0,23.7,35.0,4.6,1.0,3.0,1.0,1.0,...,23.7,36.0,4.8,1.0,2.0,3.0,0.0,0.0,53.0,0
10679,98.0,5.50,152.0,28.4,30.0,6.3,3.0,0.0,0.0,0.0,...,30.6,65.0,7.2,3.0,2.0,0.0,0.0,0.0,40.0,1
32453,96.0,5.60,43.0,20.9,20.0,4.9,1.0,0.0,0.0,0.0,...,21.6,15.0,3.6,1.0,0.0,0.0,0.0,1.0,54.0,0


<IPython.core.display.Javascript object>

In [14]:
data["L100800diff"] = np.power(data.L100800_x - data.L100800_y, 3)
data["L104600diff"] = np.power(data.L104600_x - data.L104600_y, 3)
data["L103000diff"] = np.power(data.L103000_x - data.L103000_y, 3)
data["S000300diff"] = np.power(data.S000300_x - data.S000300_y, 3)
data["L101700diff"] = np.power(data.L101700_x - data.L101700_y, 3)
data["L100700diff"] = np.power(data.L100700_x - data.L100700_y, 3)

tempclass = data.CLASS
data = data.drop(columns="CLASS")
data["CLASS"] = tempclass

<IPython.core.display.Javascript object>

In [15]:
data.head()

,L100800_x,L104600_x,L103000_x,S000300_x,L101700_x,L100700_x,FIELD_33_x,FIELD_38_x,FIELD_40_x,FIELD_31_x,...,FIELD_31,SEX,AGE,L100800diff,L104600diff,L103000diff,S000300diff,L101700diff,L100700diff,CLASS
15320,96.0,5.30,238.0,25.1,149.0,7.5,2.0,3.0,2.0,0.0,...,0.0,0.0,44.0,-1331.0,-0.000512,-1259712.0,0.000,-1.0,-0.064,1
7243,93.0,5.16,139.0,28.4,18.0,6.3,1.0,1.0,0.0,0.0,...,0.0,0.0,39.0,64.0,0.046656,-27000.0,-0.027,-4096.0,-0.008,0
32596,108.0,5.60,150.0,23.7,35.0,4.6,1.0,3.0,1.0,1.0,...,0.0,0.0,53.0,64.0,-0.001000,50653.0,0.008,-8.0,-1.728,0
10679,98.0,5.50,152.0,28.4,30.0,6.3,3.0,0.0,0.0,0.0,...,0.0,0.0,40.0,-64.0,-0.001000,185193.0,-0.729,-24389.0,0.027,1
32453,96.0,5.60,43.0,20.9,20.0,4.9,1.0,0.0,0.0,0.0,...,0.0,1.0,54.0,512.0,-0.166375,-91125.0,-0.216,-1.0,0.064,0


<IPython.core.display.Javascript object>

# 2. Downsample the majority class and upsample the minority

In [16]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

266 4034 7060


<IPython.core.display.Javascript object>

In [17]:
diabetic_test = diabetic.sample(50, random_state=randomseed)
prediabetic_test = prediabetic.sample(50, random_state=randomseed)
normal_test = normal.sample(50, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [18]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [20]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTENC(
    random_state=randomseed,
    categorical_features=[6, 7, 8, 9, 10],
    sampling_strategy="minority",
)
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 3984, 1: 3984, 0: 3984})
3984 3984 3984
(11952, 42) (11952,)


<IPython.core.display.Javascript object>

In [21]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=3, random_state=0, max_iter=300).fit(xtrain)
predlabel = kmeans.predict(xtest)

xtrain = pd.DataFrame(xtrain)
xtrain["cl"] = kmeans.labels_

xtest = pd.DataFrame(xtest)
xtest["cl"] = predlabel

<IPython.core.display.Javascript object>

In [22]:
# xtrainlabel = p.fit(xtrain, 3)

<IPython.core.display.Javascript object>

In [23]:
# xtestlabel = p.predict(xtest)

# xtrain = pd.DataFrame(xtrain)
# xtrain["cl"] = xtrainlabel

# xtest = pd.DataFrame(xtest)
# xtest["cl"] = xtestlabel

<IPython.core.display.Javascript object>

# 3. Generate the classifier models based on the selected 12 features

# 3.1.  Features : 12

# a. Random forest

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

parameters = {
    "max_features": ("auto", "sqrt", "log2"),
    "n_estimators": [10, 100, 200, 700],
    "max_depth": [2, 8, 10],
    "min_samples_split": [2, 8, 12],
    "min_samples_leaf": [2, 8, 12],
    "criterion": ["gini", "entropy"],
}

rf_clf = RandomForestClassifier(n_jobs=-1, verbose=1)
rf_clf = RandomizedSearchCV(rf_clf, parameters)
rf_clf.fit(xtrain, ytrain)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.3s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Paral

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:    1.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 700 out of 700 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:    1.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tas

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

<IPython.core.display.Javascript object>

In [25]:
rf_clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

<IPython.core.display.Javascript object>

In [26]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=500,
    max_depth=8,
    min_samples_split=5,
    min_samples_leaf=8,
    max_features="auto",
)

# # rf_12=rf_clf.best_estimator_
# rf_12 = RandomForestClassifier(
#     bootstrap=True,
#     class_weight=None,
#     criterion="gini",
#     max_depth=10,
#     max_features="log2",
#     max_leaf_nodes=None,
#     min_impurity_decrease=0.0,
#     min_impurity_split=None,
#     min_samples_leaf=2,
#     min_samples_split=12,
#     min_weight_fraction_leaf=0.0,
#     n_estimators=100,
#     n_jobs=-1,
#     oob_score=False,
#     random_state=None,
#     verbose=1,
#     warm_start=False,
# )

<IPython.core.display.Javascript object>

In [27]:
rf_12.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

<IPython.core.display.Javascript object>

In [28]:
ypred = rf_12.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.8066666666666666


<IPython.core.display.Javascript object>

In [29]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,40,10,0
Prediabetes,9,40,1
diabetes,0,9,41


<IPython.core.display.Javascript object>

In [30]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.82      0.80      0.81        50
           1       0.68      0.80      0.73        50
           2       0.98      0.82      0.89        50

    accuracy                           0.81       150
   macro avg       0.82      0.81      0.81       150
weighted avg       0.82      0.81      0.81       150



<IPython.core.display.Javascript object>

In [46]:
from sklearn import model_selection

scores = model_selection.cross_val_score(
    rf_12, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "RandomForestClassifier", scores)
)

Accuracy: 0.80 (+/- 0.01) [RandomForestClassifier] 
 [[0.80367586 0.80200501 0.80451128 0.82539683 0.80318258 0.80569514
 0.78894472 0.80820771 0.7998325  0.79731993]]


<IPython.core.display.Javascript object>

# b. XGBOOST 

In [31]:
parameters = {
    "max_depth": [2, 8, 10],
    "n_estimators": [10, 100, 700],
    "learning_rate": [0.05, 0.15, 0.25],
    "min_child_weight": [1, 3, 5],
    "gamma": [0.0, 0.2, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5],
}
xgb_clf = xgb.XGBClassifier()
xgb_clf = RandomizedSearchCV(xgb_clf, parameters, verbose=2)
xgb_clf.fit(xtrain, ytrain)
xgb_clf.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3, total=   1.7s
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3, total=   1.6s
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3 
[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.15, gamma=0.2, colsample_bytree=0.3, total=   1.6s
[CV] n_estimators=700, min_child_weight=1, max_depth=8, learning_rate=0.25, gamma=0.0, colsample_bytree=0.3 
[CV]  n_estimators=700, min_child_weight=1, max_depth=8, learning_rate=0.25, gamma=0.0, colsample_bytree=0.3, total=  20.7s
[CV] n_estimators=700, min_child_weight=1, max_depth=8, learning_rate=0.25, gamma=0.0, colsample_bytree=0.3 
[CV]  n_estimators=700, min_child_weight=1, max_depth=8, learning_rate=0.25, gamma=0.0, colsample_bytree=0.3, total=  21.3s
[CV] n_estimators=700, min_child_weight=1, max_depth=8, learning_rate=0.25, gamma=0.0, colsample_bytree=0.3 
[CV]  n_estimators=700, min_child_weight=1, max_depth=8, learning_ra

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.4min finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

<IPython.core.display.Javascript object>

In [32]:
import xgboost as xgb
from sklearn import metrics as m

xgb_model_12 = xgb.XGBClassifier(n_estimators=100,objective="binary:logistic", random_state=randomseed)
# objective="multi:softmax"
# objective="binary:logistic"
# xgb_model=xgb_clf.best_estimator_

# xgb_model=xgb_clf.best_estimator_
# xgb_model_12=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.3, gamma=0.2,
#               learning_rate=0.05, max_delta_step=0, max_depth=10,
#               min_child_weight=5, missing=None, n_estimators=700, n_jobs=1,
#               nthread=None, objective='multi:softprob', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)


<IPython.core.display.Javascript object>

In [33]:
xgb_model_12.fit(xtrain, ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

<IPython.core.display.Javascript object>

In [34]:
ypred = xgb_model_12.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.78


<IPython.core.display.Javascript object>

In [35]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,39,11,0
Prediabetes,10,38,2
diabetes,0,10,40


<IPython.core.display.Javascript object>

In [36]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.80      0.78      0.79        50
           1       0.64      0.76      0.70        50
           2       0.95      0.80      0.87        50

    accuracy                           0.78       150
   macro avg       0.80      0.78      0.78       150
weighted avg       0.80      0.78      0.78       150



<IPython.core.display.Javascript object>

In [47]:
scores = model_selection.cross_val_score(
    xgb_model_12, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "xgb_model", scores)
)

Accuracy: 0.80 (+/- 0.01) [xgb_model] 
 [[0.79699248 0.80033417 0.7978279  0.81620718 0.80067002 0.81407035
 0.78894472 0.81323283 0.80067002 0.79564489]]


<IPython.core.display.Javascript object>

# c. SVM

In [37]:
from thundersvm import SVC as svmgpu

parameters = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [1, 0.1, 0.01, 0.001, 0.0001],
    "kernel": ["linear", "rbf"],
}

SVC_clf = svmgpu()
SVC_clf2 = RandomizedSearchCV(SVC_clf, parameters, verbose=2)
SVC_clf2.fit(xtrain, ytrain)


# sorted(SVC_clf2.cv_results_.keys())

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] kernel=rbf, gamma=0.001, C=1000 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. kernel=rbf, gamma=0.001, C=1000, total=   1.3s
[CV] kernel=rbf, gamma=0.001, C=1000 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] .................. kernel=rbf, gamma=0.001, C=1000, total=   1.0s
[CV] kernel=rbf, gamma=0.001, C=1000 .................................
[CV] .................. kernel=rbf, gamma=0.001, C=1000, total=   1.0s
[CV] kernel=linear, gamma=0.1, C=0.1 .................................
[CV] .................. kernel=linear, gamma=0.1, C=0.1, total=   1.0s
[CV] kernel=linear, gamma=0.1, C=0.1 .................................
[CV] .................. kernel=linear, gamma=0.1, C=0.1, total=   1.0s
[CV] kernel=linear, gamma=0.1, C=0.1 .................................
[CV] .................. kernel=linear, gamma=0.1, C=0.1, total=   1.1s
[CV] kernel=linear, gamma=0.0001, C=0.1 ..............................
[CV] ............... kernel=linear, gamma=0.0001, C=0.1, total=   1.1s
[CV] kernel=linear, gamma=0.0001, C=0.1 ..............................
[CV] ............... kernel=linear, gamma=0.0001, C=0.1, total=   1.0s
[CV] kernel=linear, gamma=0.0001, C=0.1 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   32.1s finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=None, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovo',
                                 degree=3, gamma='auto', gpu_id=0, kernel='rbf',
                                 max_iter=-1, max_mem_size=-1, n_jobs=-1,
                                 probability=False, random_state=None,
                                 shrinking=False, tol=0.001, verbose=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['linear', 'rbf']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

<IPython.core.display.Javascript object>

In [38]:
SVC_clf2.best_estimator_

SVC(C=1000, cache_size=None, class_weight={}, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.001, gpu_id=0,
    kernel='rbf', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=False,
    random_state=None, shrinking=False, tol=0.001, verbose=False)

<IPython.core.display.Javascript object>

In [39]:
# scv_12=SVC_clf2.best_estimator_
scv_12 = svmgpu(
    C=100,
    cache_size=None,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.001,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [40]:
scv_12 = svmgpu(
    C=0.5,
    cache_size=None,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [41]:
# scv_12 = SVC(
#     C=70,
#     cache_size=200,
#     class_weight=None,
#     coef0=0.0,
#     decision_function_shape="ovr",
#     degree=3,
#     gamma="auto_deprecated",
#     kernel="linear",
#     max_iter=-1,
#     probability=True,
#     random_state=42,
#     shrinking=True,
#     tol=0.001,
#     verbose=False,
# )

<IPython.core.display.Javascript object>

In [42]:
scv_12.fit(xtrain, ytrain)

SVC(C=0.5, cache_size=None, class_weight={}, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.1, gpu_id=0,
    kernel='linear', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=False,
    random_state=42, shrinking=False, tol=0.001, verbose=False)

<IPython.core.display.Javascript object>

In [43]:
ypred = scv_12.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.7533333333333333


<IPython.core.display.Javascript object>

In [44]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,38,12,0
Prediabetes,10,32,8
diabetes,0,7,43


<IPython.core.display.Javascript object>

In [45]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.79      0.76      0.78        50
           1       0.63      0.64      0.63        50
           2       0.84      0.86      0.85        50

    accuracy                           0.75       150
   macro avg       0.75      0.75      0.75       150
weighted avg       0.75      0.75      0.75       150



<IPython.core.display.Javascript object>

In [48]:
scores = model_selection.cross_val_score(
    scv_12, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.77 (+/- 0.01) [SVC_clf] 
 [[0.7593985  0.77360067 0.78111947 0.79197995 0.78140704 0.75795645
 0.75711893 0.76633166 0.77135678 0.76046901]]


<IPython.core.display.Javascript object>